In [1]:
def timed(fn):
    from time import perf_counter
    from functools import wraps

    @wraps(fn)
    def inner(*args, **kwargs):
        start = perf_counter()
        result = fn(*args, **kwargs)
        end = perf_counter()
        elapsed = end - start

        args_ = [str(a) for a in args]
        kwargs_ = ["{0}={1}".format(k, v) for (k, v) in kwargs.items()]
        all_args = args_ + kwargs_
        args_str = ",".join(all_args)

        print("{0}({1}) took {2:.6f}s to run".format(
            fn.__name__, args_str, elapsed))

        return result
    return inner

## 1. Считаем Фибоначчи рекурсией

In [4]:
#если обернём так, то увидим много вызовов функции, поэтому лучше сделать ещё одну функцию и обернуть её

#@timed
def fibo_recursion(n):
    """Fibonacci recursion"""
    return 1 if n <= 2 else fibo_recursion(n - 1) + fibo_recursion(n - 2)


print(fibo_recursion(10))

55


In [7]:
@timed
def fib_recursive(n):
    return fibo_recursion(n)

print(fib_recursive(20))
print(fib_recursive(25))
print(fib_recursive(37))

fib_recursive(20) took 0.003351s to run
6765
fib_recursive(25) took 0.023620s to run
75025
fib_recursive(37) took 7.000843s to run
24157817


<b>Вывод: рекурсия считает долго</b>

## 2. Считаем Фибоначчи через loop

In [9]:
@timed
def fibo_loop(n):
    fibo1 = 1
    fibo2 = 1
    for x in range(3, n + 1):
        fibo1, fibo2 = fibo2, fibo2 + fibo1
    return fibo2

print(fibo_loop(10))
print(fibo_loop(36))

fibo_loop(10) took 0.000005s to run
55
fibo_loop(36) took 0.000005s to run
14930352


<b>Вывод: loop считает быстро</b>

## 3. Считаем Фибоначчи через reduce

In [14]:
@timed
def fib_reduce(n):
    from functools import reduce
    initial = (1, 0)
    fib_n = reduce(lambda prev, n: (prev[1] + prev[0], prev[0]), range(n-1), initial)
    return fib_n[0]

print(fib_reduce(36))

fib_reduce(36) took 0.000018s to run
14930352


<b>Вывод: reduce тоже считает быстро, но медленнее, чем loop</b>

### Измерим среднее время для нескольких итераций

In [9]:
def timed2(fn):
    from time import perf_counter
    from functools import wraps

    @wraps(fn)
    def inner(*args, **kwargs):
        elapsed_total = 0
        elapsed_count = 0
        for x in range(10):
            print("Running iteration {}...".format(x))
            start = perf_counter()
            result = fn(*args, **kwargs)
            end = perf_counter()
            elapsed = end - start
            elapsed_total += elapsed
            elapsed_count += 1

        args_ = [str(a) for a in args]
        kwargs_ = ["{0}={1}".format(k, v) for (k, v) in kwargs.items()]
        all_args = args_ + kwargs_
        args_str = ",".join(all_args)
        elapsed_avg = elapsed_total / elapsed_count

        print("{0}({1}) took {2:.6f}s to run".format(
            fn.__name__, args_str, elapsed_avg))

        return result
    return inner

In [10]:
@timed2
def fibo_red2(a):
    from functools import reduce
    initialiser = (1, 0)
    result = reduce(lambda prev, n: (prev[1] + prev[0], prev[0]), range(a-1), initialiser)
    return result[0]

print(fibo_red2(10))

#проблема здесь в том, что мы не можем задавать range для итерации не из функции.
#Конечно, можно сделать так: def timed(fn, count)...  for x in range(count)...
#а вызывать функцию так fibo_red2 = timed2(fibo_red2, 10)(10) , но это не гибко

Running iteration 0...
Running iteration 1...
Running iteration 2...
Running iteration 3...
Running iteration 4...
Running iteration 5...
Running iteration 6...
Running iteration 7...
Running iteration 8...
Running iteration 9...
fibo_red2(10) took 0.000008s to run
55


## 4. Несколько декораторов для функции

In [11]:
def logged(fn):
    from functools import wraps
    from datetime import datetime, timezone

    @wraps(fn)
    def inner(*args, **kwargs):
        run_dt = datetime.now(timezone.utc)
        result = fn(*args, **kwargs)
        print("{0}: called {1}".format(run_dt, fn.__name__))
        return result
    return inner


def timed(fn):
    from functools import wraps
    from time import perf_counter

    @wraps(fn)
    def inner(*args, **kwargs):
        start = perf_counter()
        result = fn(*args, **kwargs)
        end = perf_counter()
        print("{0} took {1:.6f} sec to run".format(fn.__name__, end - start))
        return result
    return inner

In [30]:
@timed
def fib(n):
    fib1 = 1
    fib2 = 1
    for x in range(3, n + 1):
        fib1, fib2 = fib2, fib1 + fib2
    return fib2

print(fib(10))


@logged
@timed
def factorial(n):
    from operator import mul
    from functools import reduce
    return reduce(mul, range(1, n + 1))


print(factorial(6))
#factorial = logged(timed(factorial)) - два декоратора на функции равнозначны такой записи, но если функция с
#рекурсией, то проверяй её поведение

fib took 0.000006 sec to run
55
factorial took 0.000016 sec to run
2020-05-12 07:26:26.961681+00:00: called factorial
720


In [29]:
#Можем положить сколько угодно декораторов
@logged
@timed
@logged
@timed
def factorial(n):
    from operator import mul
    from functools import reduce
    return reduce(mul, range(1, n + 1))

#factorial = logged(timed(logged(timed(factorial))))
print(factorial(10))

factorial took 0.000011 sec to run
2020-05-12 07:26:21.774279+00:00: called factorial
factorial took 0.000269 sec to run
2020-05-12 07:26:21.774279+00:00: called factorial
3628800
